<a href="https://colab.research.google.com/github/sofinvalery/ml-notebooks/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install mistralai
# !pip install langchain
# !pip install faiss-cpu
# !pip install wikipedia-api
# !pip install langchain-huggingface
# !pip install -U langchain-community
# !pip install -U langchain-mistralai
# !pip install wikipedia
import os
import warnings
from typing import List, Dict, Any
from mistralai import Mistral
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm, trange

warnings.filterwarnings("ignore", category=UserWarning, module="huggingface_hub.utils._token")
warnings.filterwarnings("ignore", category=UserWarning, module="tqdm.autonotebook")
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")

In [2]:
MISTRAL_API_KEY = "MISTRAL_API_KEY"
MISTRAL_MODEL = "open-mistral-nemo"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

In [3]:
def create_knowledge_base(topics: List[str]) -> FAISS:
    """
    Создает базу знаний на основе заданных тем из Wikipedia.

    Args:
        topics (List[str]): Список тем для загрузки из Wikipedia.

    Returns:
        FAISS: Векторное хранилище с загруженными и обработанными документами.

    Raises:
        Exception: Если возникла ошибка при создании базы знаний.
    """
    try:
        documents = []
        for topic in topics:
            loader = WikipediaLoader(query=topic, load_max_docs=1)
            documents.extend(loader.load())

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP
        )
        chunks = text_splitter.split_documents(documents)

        embeddings = HuggingFaceEmbeddings()
        vectorstore = FAISS.from_documents(chunks, embeddings)

        return vectorstore
    except Exception as e:
        raise Exception(f"Ошибка при создании базы знаний: {str(e)}")


def setup_qa_chain(knowledge_base: FAISS, prompt_template: str) -> RetrievalQA:
    """
    Настраивает цепочку вопросов и ответов (QA chain) с использованием Mistral AI.

    Args:
        knowledge_base (FAISS): Векторное хранилище с базой знаний.
        prompt_template (str): Шаблон промпта для генерации ответов.

    Returns:
        RetrievalQA: Настроенная цепочка вопросов и ответов.

    Raises:
        Exception: Если возникла ошибка при настройке QA chain.
    """
    try:
        os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY

        PROMPT = PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"])

        mistral_chat = ChatMistralAI(
            model=MISTRAL_MODEL,
            mistral_api_key=MISTRAL_API_KEY)

        chain_type_kwargs = {"prompt": PROMPT}
        qa = RetrievalQA.from_chain_type(
            llm=mistral_chat,
            chain_type="stuff",
            retriever=knowledge_base.as_retriever(),
            chain_type_kwargs=chain_type_kwargs)

        return qa
    except Exception as e:
        raise Exception(f"Ошибка при настройке QA chain: {str(e)}")


def answer_question(qa: RetrievalQA, question: str) -> str:
    """
    Отвечает на заданный вопрос, используя настроенную QA chain.

    Args:
        qa (RetrievalQA): Настроенная цепочка вопросов и ответов.
        question (str): Вопрос для ответа.

    Returns:
        str: Ответ на вопрос или сообщение об ошибке.
    """
    try:
        answer = qa.run(question)
        return answer
    except Exception as e:
        return f"Ошибка при обработке вопроса: {str(e)}"

In [5]:
if __name__ == "__main__":
    topics = [
        "Information security",
        "Computer security",
        "Network security",
        "Cryptography",
        "Denial-of-service attack"
    ]

    prompt_template = """Ты - эксперт по информационной безопасности. Используй следующую информацию для ответа на вопрос. Если информации недостаточно, используй свои знания, но отметь это в ответе.

    Контекст: {context}

    Вопрос: {question}

    Ответ:"""
    try:
        print("Создание базы знаний...")
        knowledge_base = create_knowledge_base(topics)

        print("Настройка системы вопросов и ответов...")
        qa_chain = setup_qa_chain(knowledge_base, prompt_template)

        print("\nВведите ваш вопрос (или 'exit' для завершения): ")
        while True:
            user_question = input("\nВопрос: ")
            if user_question.lower() == 'exit':
                break
            print(f"\nОтвет: {answer_question(qa_chain, user_question)}")

    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")

Создание базы знаний...
Настройка системы вопросов и ответов...

Введите ваш вопрос (или 'exit' для завершения): 

Вопрос: What is a buffer overflow attack, and how can it be prevented?

Ответ: A buffer overflow, also known as a buffer overrun, is a type of software vulnerability that occurs when a program, while writing data to a buffer, overruns the buffer's boundary and overwrites adjacent memory locations. This can lead to crashes, data corruption, or even code execution, making it a common target for attackers.

Here's how a buffer overflow attack works and some ways to prevent it:

1. **Attack Methodology**:
   - An attacker identifies a buffer in a program that is not properly validated or checked for size.
   - They send input data that is larger than the buffer can handle, causing it to overflow.
   - The excess data overwrites other data or code in memory, potentially altering the program's behavior or executing malicious code.

2. **Prevention Measures**:

   - **Input Valid